# Generate Recommendations for Non-Users & Streamlit

1. Top rated movies within a filter: weighted average of # of reviews and average ratings - DONE
    - Genomes: only display if above X threshold? Or add relevance score into the weighted average?
2. Item-Item recommendation if they liked movie X - TO DO 

UI elements to do:
- Biographic information for the actor, director (IMDB) (?)
- Add spell check to actor names 

UI in the future: 
- Filtering within personalized recommendations (user ID field)
- Tab allowing users to input own ratings and get a recommendation out
- EDA

#### To Run:
1. Convert notebook to py file
    - Run in command line: py -m jupyter nbconvert --to script streamlit_example.ipynb
2. Run streamlit app
    - Run in command line: streamlit run streamlit_example.py

In [1]:
import pandas as pd
import os
import numpy as np
from scipy.sparse import csc_matrix
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import operator
import streamlit as st
import fastparquet
import re

## Get Unique Lists of Filter Options

In [2]:
def cat_list_expand(df, var):
    
    # expand lists such that one entry per row 
    expanded = df[[var, 'movieId']]
    expanded = pd.DataFrame({
        col:np.repeat(expanded[col].values, expanded[var].str.len()) for col in expanded.columns.drop(var)}
    ).assign(**{var:np.concatenate(expanded[var].values)})[expanded.columns]

    return expanded

In [3]:
def unique_lists(df):
    
    # unique lists. Sort alphabetically
    genres_unique = np.sort(cat_list_expand(df, 'Genres').Genres.unique())
    actors_unique = np.sort(cat_list_expand(df, 'actors_downcased').actors_downcased.unique())
    directors_unique = np.sort(cat_list_expand(df, 'directors_downcased').directors_downcased.unique())
    countries_unique  = np.sort(cat_list_expand(df, 'Filming Countries')['Filming Countries'].unique())
    language_unique = np.sort(cat_list_expand(df, 'Language(s)')['Language(s)'].unique())
    tags_unique = np.sort(cat_list_expand(df, 'Tags').Tags.unique())
    
    return genres_unique, actors_unique, directors_unique, countries_unique, language_unique, tags_unique

# Run above functions w. cacheing
These functions will not change when user inputs to filtering change, so only run first time app is opened     
st.cache not compatabile with jupyter notebook. Comment out when running in jupyter. 

In [5]:
@st.cache
def cached_functions():
    
    # read in data created in recommendation_data_display.ipynb
    df = pd.read_parquet('recommendation_display.parq')

    # get unique lists of all filter values
    genres_unique, actors_unique, directors_unique, countries_unique, language_unique, tags_unique = unique_lists(df)
    
    return df, genres_unique, actors_unique, directors_unique, countries_unique, language_unique, tags_unique

In [6]:
df_display, genres_unique, actors_unique, directors_unique, countries_unique, language_unique, tags_unique = cached_functions()

## Display in Streamlit with filter options
Display:
- Title
- Year
- Description
- Duration
- Genres
- Actors
- Directors
- Production Company
- Country
- Language
- Genome Tags
- Number of ratings
- Average rating    
   
Filter by:
- Genres
- Actors
- Directors
- Country
- Language
- Genome Tags

Default table is highest rated movies without filters    
   
   
Extensions:
- AND/OR advanced search option? This might be difficult

In [7]:
st.title('Top Rated Movie Recommendations')
st.header('Enter desired filters and select "Display Recommendations" \n')
st.write('Please note filters use AND logic')
st.write('If you wish to see overall top rated movies, select Display Recommendations without any filters')

In [8]:
# get user inputs: multiple selection possible per category
genre_input = st.multiselect('Select genre(s)', genres_unique)
country_input = st.multiselect('Select country(s)', countries_unique)
language_input = st.multiselect('Select language(s)', language_unique)
tag_input = st.multiselect('Select genome tags(s)', tags_unique)

# actors, directors get text inputs
# Dropdowns too much for streamlit to handle
# allow multiple entires
actor_input = st.text_input('Type actor(s) names separated by comma')
if actor_input != '':
    # downcase input
    actor_input = actor_input.lower()
    # split into list 
    actor_input = actor_input.split(', ')
    # check valid actor/in dataframe. Else, notify and do not apply filter
    for i in actor_input:
        if i not in actors_unique:
            st.write('Invalid actor', i)
            actor_input = []
else:
    actor_input = []

director_input = st.text_input('Type director(s) names separated by comma')
if director_input != '':
    # downcase input
    director_input = director_input.lower()
    # split into list
    director_input = director_input.split(', ')
    # check valid director/in dataframe. Else, notify and do not apply filter
    for i in director_input:
        if i not in directors_unique:
            st.write('Invalid director', i)
            director_input = []
else:
    director_input = []

# display recommendations once hit button
if st.button('Display Recommendations'):
    # filter dataframe
    df_filtered = df_display[(df_display.Genres.map(set(genre_input).issubset)) & 
                             (df_display['Filming Countries'].map(set(country_input).issubset)) &
                             (df_display['Language(s)'].map(set(language_input).issubset)) & 
                             (df_display.Tags.map(set(tag_input).issubset))  & 
                             (df_display.actors_downcased.map(set(actor_input).issubset)) &
                             (df_display.directors_downcased.map(set(director_input).issubset))
                            ].sort_values('weighted_avg', ascending = False).head(10).drop(columns = ['weighted_avg',
                                                                                                     'actors_downcased', 
                                                                                                      'directors_downcased',
                                                                                                     'title_downcased', 
                                                                                                     'title_year'])
    # if no valid movies with combination of filters, notify. Else display dataframe
    if len(df_filtered) > 0:
        st.write(df_filtered)
    else:
        st.write('Found no movies that match your selections')